<a href="https://colab.research.google.com/github/gmshroff/metaLearning2022/blob/main/code/nb0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TRAINING DEEP NEURAL NETWORKS

In [ ]:
# !pip install import_ipynb --quiet
# !git clone https://github.com/gmshroff/metaLearning.git
# %cd metaLearning/code

In [ ]:
import import_ipynb
import utils
import models
from IPython import display
import torch
from sklearn.manifold import TSNE
from matplotlib import pyplot as plt
import pickle

In [ ]:
from course_data import MyDS

Random data in $\mathcal{R}^N$ in $C$ clusters/classes.

In [ ]:
#Generate data - euclidean
train_ds, test_ds, dloader = utils.euclideanDataset(n_samples=10000,n_features=20,n_classes=10,batch_size=32)

Using random data in $\mathcal{R}^4$ as above to generate sin wave mixures; each class has different mean frequencies and amplitues.

Project data down to a 2-D space for visualiation.

In [ ]:
tsne = TSNE()
X_2d = tsne.fit_transform(train_ds.samples[0:100])
colors=["red","green","blue","yellow","pink","black","orange","purple","beige","brown"]
plt.scatter(X_2d[:,0],X_2d[:,1],c=[colors[l] for l in train_ds.labels[0:100]])
plt.show()

Simple MLP on Eucledian Data

In [ ]:
# Define an MLP network. Note that input dimension has to be data dimension. For classification
# final dimension has to be number of classes; for regression one.
# torch.manual_seed(10)
net = models.MLP(dims=[20,128,64,10])

In [ ]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,losses,accs=models.Train(net,dloader,epochs=10,verbose=True)

Accuracy on training set.

In [ ]:
#Training accuracy.
models.accuracy(net,train_ds.samples,train_ds.labels,verbose=True)

Accuracy on test set.

In [ ]:
# Test accuracy.
models.accuracy(net,test_ds.samples,test_ds.labels)

The well-known MNIST data of handwritten characters.

In [ ]:
train_ds, test_ds, dloader = utils.mnist_data(frac=0.25,download=True)

In [ ]:
plt.imshow(train_ds.samples.reshape(15000,28,28)[10])
plt.show()

In [ ]:
train_ds.samples[0].shape

In [ ]:
# Define an MLP network. Note that input dimension has to be data dimension. For classification
# final dimension has to be number of classes; for regression one.
#torch.manual_seed(10)
net = models.MLP(dims=[784,128,64,10])

In [ ]:
net(train_ds.samples[10].unsqueeze(0))

In [ ]:
# Train the network; note that network is trained in place so repeated calls further train it.
net,losses,accs=models.Train(net,dloader,epochs=10,verbose=True)

Train the network

In [ ]:
#Training accuracy.
models.accuracy(net,train_ds.samples,train_ds.labels,verbose=True)

In [ ]:
# Test accuracy.
models.accuracy(net,test_ds.samples,test_ds.labels)